<a href="https://colab.research.google.com/github/AlvinKimata/transformers/blob/main/examples/pytorch/language-modeling/llama_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           334Gi       2.0Gi       316Gi       2.0Mi        16Gi       330Gi
Swap:             0B          0B          0B


In [ ]:
%%capture
!pip install torch~=2.4.0 torch_xla[tpu]~=2.4.0 -f https://storage.googleapis.com/libtpu-releases/index.html
!git clone -b llama2-google-next-training https://github.com/pytorch-tpu/transformers.git
%cd transformers
!pip install peft
!pip install -e . --user
!pip install datasets accelerate evaluate scikit-learn
!pip uninstall -y tensorflow jax

In [5]:
!wget https://raw.githubusercontent.com/AlvinKimata/transformers/main/examples/pytorch/language-modeling/run_clm.py

--2024-08-21 10:21:47--  https://raw.githubusercontent.com/AlvinKimata/transformers/main/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37868 (37K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  36.98K  --.-KB/s    in 0.01s   

2024-08-21 10:21:47 (3.44 MB/s) - ‘run_clm.py’ saved [37868/37868]



In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
# Run
!python run_clm.py \
  --tokenizer_name hf-internal-testing/llama-tokenizer \
  --dataset_name wikitext \
  --dataset_config_name wikitext-2-raw-v1 \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --num_train_epochs 2 \
  --do_train \
  --output_dir /tmp/output \
  --overwrite_output_dir \
  --model_name_or_path "meta-llama/Meta-Llama-3-8B"\
  --save_strategy no \
  --logging_strategy no \
  --remove_unused_columns no \
  --optim adafactor \
  --torch_dtype bfloat16 \
  --dataloader_drop_last yes \
  --block_size 256 \
  --spmd_2d_sharding 0 \
  --spmd_grad_chkpt True \
  --spmd_fsdp_sharding True \
  --peft_lora True

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
checkpoint_manager_path=None,
data_seed=None,
dataloader_drop_last=True,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fs